In [1]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, PrimaryKeyConstraint, UniqueConstraint, ForeignKeyConstraint
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

In [2]:
engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class Program(DeclBase):
    __tablename__ = 'programs'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    program_courses = relationship("Programs_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program
        
class Course(DeclBase):
    __tablename__ = 'courses'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Programs_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name 
        
class Programs_course(DeclBase):
    __tablename__ = 'programs_courses'
    __table_args__ = {'extend_existing': True}
    __table_args__ = (PrimaryKeyConstraint('semester_number', 'course_id', 'program_id'),)
   
    semester_number = Column(Integer)
    course_id = Column(Integer, ForeignKey('courses.id'))
    program_id = Column(Integer, ForeignKey('programs.id'))
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    __table_args__ = {'extend_existing': True}
    __table_args__ = (PrimaryKeyConstraint('student_id', 'course_id'),)
    mark = Column(Integer)
    
    student_id = Column(Integer, ForeignKey('students.id'))
    course_id = Column(Integer, ForeignKey('courses.id'))
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [4]:
print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Антон
-  Василий
Программа:  теоретическая физика
-  Виолетта
-  Исидора
Программа:  Программная инженерия
-  Пётр


In [5]:
co = Course("Нанобиотехнологии")
p = Program("Биоинформатика")

pc1 = Programs_course("2", co, p)

session.add(pc1)
try:
    session.commit()
except:
    raise
finally:
    session.close()

In [6]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.surname, s.patronymic, s.card)
    for pc in p.program_courses:
        print("- ", pc.program_id, pc.course_id, pc.semester_number)
session.commit()

Программы и студенты
Программа:  математика и информационные технологии
-  Антон Битов None 180101
-  Василий Пупкин None 199099
Программа:  теоретическая физика
-  Виолетта Аргонова None 180201
-  Исидора Петрова Ивановна 002003
Программа:  Программная инженерия
-  Пётр Иванов Сидорович 002002
Программа:  Биоинформатика
-  4 1 2


In [7]:
print("Курсы")
for p in session.query(Course):
    print("Курс: ", p.name)

Курсы
Курс:  Нанобиотехнологии


In [8]:
pr = session.query(Program).filter_by(name="Биоинформатика")[0]
st = session.query(Student).filter_by(name="Виолетта")[0]

st.program = pr
session.commit()

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Антон
-  Василий
Программа:  теоретическая физика
-  Исидора
Программа:  Программная инженерия
-  Пётр
Программа:  Биоинформатика
-  Виолетта


In [9]:
co = Course("Нанотехнологии")
p = Program("теоретическая физика")

pc1 = Programs_course("4", co, p)

session.add(pc1)
try:
    session.commit()
except:
    raise
finally:
    session.close()

In [10]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.surname, s.patronymic, s.card)
    for pc in p.program_courses:
        print("- ", pc.program_id, pc.course_id, pc.semester_number)
session.commit()

Программы и студенты
Программа:  математика и информационные технологии
-  Антон Битов None 180101
-  Василий Пупкин None 199099
Программа:  теоретическая физика
-  Исидора Петрова Ивановна 002003
Программа:  Программная инженерия
-  Пётр Иванов Сидорович 002002
Программа:  Биоинформатика
-  Виолетта Аргонова None 180201
-  4 1 2
Программа:  теоретическая физика
-  5 2 4


In [11]:
print("Курсы")
for p in session.query(Course):
    print("Курс: ", p.name, p.id)

Курсы
Курс:  Нанобиотехнологии 1
Курс:  Нанотехнологии 2


In [12]:
st = Student("006006", "Михайлов", "Александр", "Ильич", Program("Экономика"))

m1 = Mark("5", st, Course("Нанотехнологии"))

session.add(m1) 
session.commit()

In [13]:
print("Студенты и оценки: ")
for p in session.query(Student):
    print("Студент: ", p.name)
    for pc in p.marks:
        print("- ", pc.student_id, pc.course_id, pc.mark)
session.commit()

Студенты и оценки: 
Студент:  Антон
Студент:  Виолетта
Студент:  Василий
Студент:  Пётр
Студент:  Исидора
Студент:  Александр
-  6 3 5
